## Chatbot to take inputs from a user and translate into sql script to query the database

### Import libraries

In [2]:
import sqlite3
import os
from fastapi import FastAPI
from pydantic import BaseModel

### Query the stock.db to pull rows from respective tables

- Table: signals(stock, signal_type, confidence, date)
- Table: pnl(stock, profit, loss, date)
- Table: top_indices(index_name, performance, date)
- Table: news(stock, headline, date, sentiment)

In [4]:
os.getcwd()

'c:\\Gen_ai_projects\\navya_chatbot\\stock_chatbot\\stockbot\\app'

In [5]:
DB_PATH = os.path.join(os.getcwd(), 'stock.db')
DB_PATH = os.path.abspath(DB_PATH)

print(DB_PATH)

c:\Gen_ai_projects\navya_chatbot\stock_chatbot\stockbot\app\stock.db


In [7]:
from sql_handler import run_sql_query

query = "SELECT * FROM signals WHERE date = '2025-07-22';"
results = run_sql_query(query)
print(results)

[{'stock': 'CG', 'signal_type': 'BUY', 'confidence': 0.85, 'date': '2025-07-22'}, {'stock': 'NIMB', 'signal_type': 'SELL', 'confidence': 0.78, 'date': '2025-07-22'}, {'stock': 'SBIN', 'signal_type': 'HOLD', 'confidence': 0.5, 'date': '2025-07-22'}]


### Generate SQL query from normal text using Ollama framework and Mistral foundation model

In [12]:
# import sys
# import os
# sys.path.append(os.path.abspath('c:/Gen_ai_projects/navya_chatbot/stock_chatbot/stockbot/app'))

from prompt_template import SQL_GEN_TEMPLATE, ANSWER_GEN_TEMPLATE
from llm_client import call_ollama_to_generate_sql, explain_sql_result_ollama


In [13]:
question = "What is the signal for CG on 2025-07-22?"
result = call_ollama_to_generate_sql(question)
print("Generated SQL:\n", result)

Prompt sent to model:
 
You are a helpful assistant. Convert the user question to an SQL query based on the following schema:

- Table: signals(stock, signal_type, confidence, date)
- Table: pnl(stock, profit, loss, date)
- Table: top_indices(index_name, performance, date)
- Table: news(stock, headline, date, sentiment)

Question: What is the signal for CG on 2025-07-22?
SQL:

Generated SQL:
 SELECT signal_type, confidence
FROM signals
WHERE stock = 'CG' AND date = '2025-07-22';


### Use the generated SQL query to pull relevant information from the respective table and schema

In [14]:
from llm_client import call_ollama_to_generate_sql, explain_sql_result_ollama
from prompt_template import SQL_GEN_TEMPLATE, ANSWER_GEN_TEMPLATE
from sql_handler import run_sql_query

In [16]:
question = "What is the signal for CG on 2025-07-22?"
sql_query = call_ollama_to_generate_sql(question)
table_output = run_sql_query(sql_query)
print(table_output)

Prompt sent to model:
 
You are a helpful assistant. Convert the user question to an SQL query based on the following schema:

- Table: signals(stock, signal_type, confidence, date)
- Table: pnl(stock, profit, loss, date)
- Table: top_indices(index_name, performance, date)
- Table: news(stock, headline, date, sentiment)

Question: What is the signal for CG on 2025-07-22?
SQL:

[{'signal_type': 'BUY', 'confidence': 0.85}]


### Present the generated result in normal english

In [18]:
# Step 2: Execute SQL
natural_response = explain_sql_result_ollama(question, table_output)
print({"user_query": question,
        "generated_sql": sql_query,
        "result": table_output,
        "natural_response": natural_response
})

{'user_query': 'What is the signal for CG on 2025-07-22?', 'generated_sql': "SELECT signals.signal_type, signals.confidence\nFROM signals\nWHERE signals.stock = 'CG' AND signals.date = '2025-07-22';", 'result': [{'signal_type': 'BUY', 'confidence': 0.85}], 'natural_response': "On July 22, 2025, the signal for CG is a BUY signal. The confidence level of this prediction is quite high at 85%. This means there's a strong likelihood that it could be a good time to invest in CG shares if you follow financial market recommendations based on signals like these. However, remember that investing always carries risk and it's important to do your own research before making any investment decisions."}


In [19]:
print(natural_response)

On July 22, 2025, the signal for CG is a BUY signal. The confidence level of this prediction is quite high at 85%. This means there's a strong likelihood that it could be a good time to invest in CG shares if you follow financial market recommendations based on signals like these. However, remember that investing always carries risk and it's important to do your own research before making any investment decisions.
